In [ ]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import re
from geneva_stroke_unit_preprocessing.utils import create_ehr_case_identification_column

In [ ]:
data_path = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20220815'
drugs_file_start = 'prescription'

In [ ]:
drugs_files = [pd.read_csv(os.path.join(data_path, f), delimiter=';', encoding='utf-8')
               for f in os.listdir(data_path)
               if f.startswith(drugs_file_start)]

In [ ]:
prescription_df = pd.concat(drugs_files, ignore_index=True)

In [ ]:
prescription_df['case_admission_id'] = create_ehr_case_identification_column(prescription_df)

In [ ]:
columns_to_drop = ['nr', 'patient_id', 'eds_end_4digit', 'eds_manual', 'DOB', 'begin_date',
                   'end_date', 'death_date', 'death_hosp', 'eds_final_id',
                   'eds_final_begin', 'eds_final_end', 'eds_final_patient_id',
                   'eds_final_birth', 'eds_final_death', 'eds_final_birth_str',
                   'date_from', 'date_to']
prescription_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
prescription_df.head()

In [ ]:
# find all drugs with "mmHg" in the column long_name
prescription_df[prescription_df.long_name.str.contains('mmHg') == True].long_name.value_counts()

In [ ]:
prescription_df[prescription_df.long_name.str.contains('mmHg') == True]

In [ ]:
prescription_df[prescription_df.frequency_label == "ordre unique"]

# Identify BP strategies

1. identify prescriptions related to BP strategies
2. identify targeted variable: TAM / TAS / TA (TAS + TAD)
3. identify targeted value: X mmHg
4. identify condition: target range, vs intervention threshold

## 1. identify prescriptions related to BP strategies

- find BP goal identifiers in long_name
- find meds given en reserve

In [ ]:
mmHg_equivalents = ['mmHg', 'mm Hg', 'mmg Hg', 'mmhHg', 'mHg']
mmHg_equivalents_upper = [s.upper() for s in mmHg_equivalents]

bp_goal_identifiers = [' TAM ', ' TAS ', ' TA '] + mmHg_equivalents
bp_goal_identifiers_upper = [s.upper() for s in bp_goal_identifiers]

In [ ]:
prescription_df[(prescription_df.long_name.str.upper().str.contains('|'.join(bp_goal_identifiers).upper()) == True)].long_name.value_counts()

Notes: condition is delimited by <b> and </b> and contains bp_goal_identifiers

In [ ]:
long_name_parts = prescription_df[(prescription_df.long_name.str.upper().str.contains('|'.join(bp_goal_identifiers).upper()) == True)].long_name.str.upper().str.split('<B>')

In [ ]:
bp_prescriptions_name_and_date_df = prescription_df[(prescription_df.long_name.str.upper().str.contains('|'.join(bp_goal_identifiers).upper()) == True)][['case_admission_id','short_name', 'start_date', 'end_date.1', 'stop_date']]

In [ ]:
def find_element_with_string_in_list(main_list, search_strings):
    for element in main_list:
        for search_string in search_strings:
            if search_string in element:
                return element
    return None

In [ ]:
# select the long_name_parts containing a string containg bp_goal_identifiers (upper case)
conditions = long_name_parts.apply(lambda x: find_element_with_string_in_list(x, bp_goal_identifiers_upper))

In [ ]:
conditions

## 2. identify targeted variable: TAM / TAS / TA (TAS/TAD)

In [ ]:
def target_variable_identification_scheme(condition):
    start_regex = '(?:^| |>|&GT|\(|,)'
    end_regex = '(?= |$|>|<|&GT|&LT|&NBSP|\)|1|,)'
    if condition is None:
        return None
    
    # if ' TAM' followed by a space or a > or a < or the end of the string
    TAM_equivalents = ['TAM', 'TENSION ARTÉRIELLE MOYENNE', 'PAM', 'TAMOYENNE', 'TENSION MOYENNE', 'TASM', 'TSAM', 'TA M']
    TAM_rgx = rf'{start_regex}({"|".join(TAM_equivalents)}){end_regex}'
    if re.search(TAM_rgx, condition) is not None:
        return 'TAM'
    
    # if a number in this format XXX/XX or XXX/XXX is given, return 'TA'
    if re.search(r'\d{2,3}/\d{2,3}', condition) is not None:
        return 'TA'
    
    TAS_equivalents = ['TAS', 'TENSION ARTÉRIELLE SYSTOLIQUE', 'PAS', 'TASYS', 'SISOTLIQUE', 'TENSION SYSTOLIQUE', 
                       'TASYSTOLIQUE', 'SYSTOLIQUE', 'HTAS', 'SYSTOLE', 'SISTOLIQUE', 'SISTOLE', 'SYSTOLES']
    TAS_rgx = rf'{start_regex}({"|".join(TAS_equivalents)}){end_regex}'
    if re.search(TAS_rgx, condition) is not None:
        return 'TAS'
    
    # if only one value provided for target, use as TAS (two variable target interpreted as TA above)
    TA_equivalents = ['TA', 'PA']
    TAS_rgx = rf'{start_regex}({"|".join(TA_equivalents)}){end_regex}'
    if re.search(TAS_rgx, condition) is not None:
        return 'TAS'
    
    # if there is a single 2 or 3 digit number, if number > 110, return 'TAS', else return 'TAM'
    target_pressure_matches = re.findall(r'\d{2,3}', condition)
    if len(target_pressure_matches) > 0:
        # take the first number found (because no better rule yet)
        if int(target_pressure_matches[0]) > 110:
            return 'TAS'
        else:
            return 'TAM'
    
    else:
        return "unknown"
    

In [ ]:
target_variables = conditions.apply(lambda x: target_variable_identification_scheme(x))

In [ ]:
target_variables.value_counts()

In [ ]:
pd.DataFrame([conditions, target_variables]).T

## 3. identify targeted value: X mmHg

In [ ]:
def target_value_identification_scheme(condition):
    if condition is None:
        return None
    
    # patch common mistypings
    condition = condition.replace('1600', '160')
    condition = condition.replace('2201', '220')
    condition = condition.replace('1140', '140')
    condition = condition.replace('995', '95')
    condition = condition.replace('22O', '220')
    
    # if there is a pattern such as XXX/XXX [mmHg equivalent] or XXX/XX [mmHg equivalent] (with or without spaces)
    # return the two numbers
    mmHg_equivalents_rgx = "|".join(mmHg_equivalents_upper)
    target_pressure_matches = re.search(rf'(\d{{2,3}})/(\d{{2,3}})( |)({mmHg_equivalents_rgx})', condition)
    if target_pressure_matches is not None:
        target_pressure_string = re.search(r'(\d{2,3})/(\d{2,3})', target_pressure_matches[0])
        if target_pressure_string is not None:
            return target_pressure_string[0]
    
    # check for XXX/XXX or XXX/XX pattern
    target_pressure_matches = re.findall(r'(\d{2,4})/(\d{2,3})', condition)
    if len(target_pressure_matches) > 0:
        value1 = target_pressure_matches[0][0]
        value2 = target_pressure_matches[0][1]
        
        # if the first number has 4 digits, divide by 10
        if len(value1) == 4:
            return f'{int(value1)[:-1]}/{value2}'
        
        # if the second number > first number, swap
        if int(value2) > int(value1):
            temp_first = value1
            value1 = value2
            value2 = temp_first
            
        # to safeguard against other numbers presenting as XX/XX, TAS should be > 75
        if int(value1) > 75:
            return f'{value1}/{value2}'
            
        
    # if there is a single 2 or 3 digit number, with a leading > or <, return the number (without space in between)
    target_pressure_matches = re.findall(r'(<|<, OU =|INFÉRIEURE À|>|&GT|&LT|SUPERIEUR A|SUPÉRIEURE À|SUPÉRIEUR À|SUP À|SUP&NBSP, À| PLUS DE|>, OU =|PLUS QUE)(,|)(| |,)(| |,)(\d{2,3})', condition)
    # filter > pertaining to B> or I>
    target_pressure_matches = [m for m in target_pressure_matches 
                               if not (m[0] == '>' and 
                                  (condition.split(''.join(m))[0][-1] == 'I'
                                   or condition.split(''.join(m))[0][-1] == 'B'))]
    if len(target_pressure_matches) > 0:
        # take the first number found (because no better rule yet)
        if int(target_pressure_matches[0][-1]) >= 50: 
            return target_pressure_matches[0][-1]

    # identify a range
    target_ranges_matches = re.search(rf'(\d{{2,3}})( |)({mmHg_equivalents_rgx}|)(| )(-|ET|ET <,|ET &LT,|AU MAXIUMUM<BR>AU MIN)(| )(\d{{2,3}})( |)({mmHg_equivalents_rgx}|)', condition)
    if target_ranges_matches is not None:
        target_range_string1 = re.search(r'(\d{2,3})-(\d{2,3})', target_ranges_matches[0])
        if target_range_string1 is not None:
            return target_range_string1[0]
        target_range_string2 = re.findall(r'(\d{2,3})', target_ranges_matches[0])
        if len(target_range_string2) > 0:
            if (int(target_range_string2[1]) > int(target_range_string2[0]))\
                    and (int(target_range_string2[1]) > 65):
                return f'{target_range_string2[0]}-{target_range_string2[1]}'
            elif int(target_range_string2[0]) > 65:
                return f'{target_range_string2[1]}-{target_range_string2[0]}'

    
    # if there is a single 2 or 3 digit number
    target_pressure_matches = re.findall(r'\d{2,3}', condition)
    if len(target_pressure_matches) > 0:
        # take the first number found (because no better rule yet)
        
        # excluded instances where match is followed by 'MG'
        rest_condition = condition.split(target_pressure_matches[0])[-1]
        if rest_condition.startswith('MG') or rest_condition.startswith(' MG'):
            return "unknown"
        
        if int(target_pressure_matches[0]) > 50:
            return target_pressure_matches[0]
    
  
    return "unknown"

In [ ]:
target_values = conditions.apply(lambda x: target_value_identification_scheme(x))


In [ ]:
target_values.value_counts()

In [ ]:
pd.DataFrame([conditions, target_variables, target_values]).T

## 4. identify condition: > / < / range (-)

In [ ]:
def target_condition_identification_scheme(condition):
    if condition is None:
        return None
    
    # identify a range
    mmHg_equivalents_rgx = "|".join(mmHg_equivalents_upper)
    target_ranges_matches = re.search(rf'(\d{{2,3}})( |)({mmHg_equivalents_rgx}|)(| )(-|ET|ET <,|ET &LT,|AU MAXIUMUM<BR>AU MIN)(| )(\d{{2,3}})( |)({mmHg_equivalents_rgx}|)', condition)
    if target_ranges_matches is not None:
        return 'range'
           
    # if there is a single 2 or 3 digit number, with a leading  < (or equivalent), return <
    inferior_than_equivalents = ['<', '<, OU =', 'INFÉRIEURE À','&LT', '&LT, OU = À']
    inferior_than_equivalents_rgx = '|'.join(inferior_than_equivalents) 
    inferior_than_matches = re.findall(rf'({inferior_than_equivalents_rgx})(,|)(| |,)(| |,)(\d{{2,3}})', condition)
    if len(inferior_than_matches) > 0:
        if int(inferior_than_matches[0][-1]) >= 50: 
            return '<'
    
    # if there is a single 2-3 digit number, with a leading >, return > 
    superior_than_equivalents = ['>', '&GT', 'SUPERIEUR A', 'SUPÉRIEURE À', 'SUPÉRIEUR À', 'SUP À', 'SUP&NBSP, À', ' PLUS DE', '>, OU =', 'PLUS QUE', 'AU DESSUS', 'AU-DESSUS', 'SUPÉRIEURE AUX', 'SUPERIEUR À']
    superior_than_equivalents_rgx = '|'.join(superior_than_equivalents)
    superior_than_matches = re.findall(rf'({superior_than_equivalents_rgx})(,|)(| |,)(| |,)(\d{{2,3}})', condition)
    superior_than_matches = [m for m in superior_than_matches 
                               if not (m[0] == '>' and 
                                  (condition.split(''.join(m))[0][-1] == 'I'
                                   or condition.split(''.join(m))[0][-1] == 'B'))]
    if len(superior_than_matches) > 0:
        if int(superior_than_matches[0][-1]) >= 50: 
            return '>'
  
    # identify isolated < or > 
    # remove isolated <I>, </I>, <B>, </B>, as well as I> and B>
    to_replace = ['<I>', '</I>', '<B>', '</B>', 'I>', 'B>']
    for r in to_replace:
        condition = condition.replace(r, '')
    
    any_inferior_than_matches = re.findall(rf'({inferior_than_equivalents_rgx})', condition)
    if len(any_inferior_than_matches) > 0:
        return '<'
    
    any_superior_than_matches = re.findall(rf'({superior_than_equivalents_rgx}|MAX|EN RÉSERVE SI)', condition)
    if len(any_superior_than_matches) > 0:
        return '>'
  
    return "unknown"

In [ ]:
target_conditions = conditions.apply(lambda x: target_condition_identification_scheme(x))


In [ ]:
target_conditions.value_counts()

In [ ]:
conditions[target_conditions == 'unknown']

In [ ]:
bp_targets_df = bp_prescriptions_name_and_date_df.join(pd.DataFrame([conditions, target_variables,target_conditions, target_values]).T)
bp_targets_df.columns = ['case_admission_id', 'short_name', 'start_date', 'end_date', 'stop_date', 'condition', 'target_variable', 'target_condition', 'target_value']

In [ ]:
bp_targets_df

In [ ]:
bp_targets_df.short_name.value_counts()

# filter db for BP strategies

- remove nimodipine, isosorbide dinitrate, sacubitril + valsartan, altéplase, sacubitril + valsartan Entresto cp, insuline aspart, midazolam, clopidogrel
- si contiens cp, only keep first word
- filter out if long_name contains "NE PAS DONNER"
- filter out if condition contains multiple dates

In [ ]:
bp_targets_df = bp_targets_df[~bp_targets_df.short_name.isin(['nimodipine', 'isosorbide dinitrate', 'sacubitril + valsartan', 'altéplase', 'sacubitril + valsartan Entresto cp', 'insuline aspart', 'midazolam', 'clopidogrel'])]
bp_targets_df = bp_targets_df[~bp_targets_df.condition.str.contains('NE PAS DONNER')]

In [ ]:
def remove_pharma_drug_name(x):
    word_list = x.split(' ')
    if ('cp' in x) or ('inject' in x):
        if len(word_list) < 4:
            return word_list[0]
        elif len(word_list) < 6:
            return ' '.join(word_list[:3])
        else:
            print(x)
            return ' '.join(word_list[:3])
        
    else:
        return x
        

In [ ]:
def ensure_correct_order_for_range(x):
    # check if '-' in x
    if not '-' in x:
        return x
    
    # check if first number is smaller than second number
    numbers = x.split('-')
    if int(numbers[0]) < int(numbers[1]):
        return x
    else:
        return f'{numbers[1]}-{numbers[0]}'

In [ ]:
def ensure_correct_order_for_sbp_dbp(x):
    # check if '/' in x
    if not '/' in x:
        return x
    
    # check if first number is greater than second number
    numbers = x.split('/')
    if int(numbers[0]) > int(numbers[1]):
        return x
    else:
        return f'{numbers[1]}/{numbers[0]}'

In [ ]:
# filter out instructions with multiple dates
def count_dates_occurrences(s):
    s = s.condition
    pattern = re.compile(r'\b\d{2}\.\d{2}\b')
    matches = pattern.findall(s)
    
    if len(matches) > 1:
        return True
    else:
        return False

In [ ]:
bp_targets_df.short_name = bp_targets_df.short_name.apply(lambda x: remove_pharma_drug_name(x))
bp_targets_df.target_value = bp_targets_df.target_value.apply(lambda x: ensure_correct_order_for_range(x))
bp_targets_df.target_value = bp_targets_df.target_value.apply(lambda x: ensure_correct_order_for_sbp_dbp(x))
bp_targets_df = bp_targets_df[~bp_targets_df.filter(like='condition', axis=1).apply(count_dates_occurrences, axis=1)]

## Seperate anti-hypertensive drugs from vasopressors/fluids

In [ ]:
drug_class_lists_df = pd.read_excel('./drug_class_short_names.xlsx')

## Antihypertensive strategies

In [ ]:
antihypertensives_bp_targets_df = bp_targets_df[bp_targets_df.short_name.isin(drug_class_lists_df.antihypertensive)]
antihypertensives_bp_targets_df

In [ ]:
antihypertensives_bp_targets_df[antihypertensives_bp_targets_df.target_condition == 'range'].target_value.value_counts()

#### Strategies 


##### SBP Strategies
- SBP < 130 mmHg: 130/80-85 (n~= 30)
+/- = MAP < 100 mmHg

- SBP < 140 mmHg: 140/80-105, or 120-140 or 130-140 (n~=1500)
+/- = MAP < 100-117 mmHg

- SBP < 150 mmHg: 150/80-110 (n~=900)
+/- = MAP < 103-123 mmHg

- SBP < 160 mmHg: 160/80-120 or 140-160 (n~=900)
+/- = MAP < 106-133 mmHg

- SBP < 180 mmHg: 180/90-120 (n~=800)
+/- = MAP < 120-140 mmHg

- SBP < 220 mmHg: 200/105-120 (n~=500)
+/- = MAP < 143-153 mmHg


##### MAP Strategies
- MAP < 100 mmHg (n~=32)
- MAP < 105 mmHg (n~=950)
- MAP < 110 mmHg (n~=64)
- MAP < 115 mmHg (n~=390)
- MAP < 120 mmHg (n~=14)
- MAP < 130 mmHg (n~=800)
- MAP < 150 mmHg (n~=55)

Note: pathphys, MAP makes more sense, but evidence built around SBP


##### Strategy labels:

- 0: MAP < 105 mmHg
- 1: SBP < 140 mmHg
- 2: SBP < 150 mmHg
- 3: SBP < 160 mmHg
- 4: MAP < 130 mmHg
- 5: SBP < 180 mmHg
- 6: SBP < 220 mmHg 
- 7: not limited



In [ ]:
def target_strategy_identification(x):
    # strategy 0: MAP < 105 mmHg
    if any([
            (x.target_variable == 'TAM') & (x.target_value == '105'),
            ('-' in x.target_value) & (x.target_value.endswith('105'))
    ]):
        return 0
    
    # strategy 1: SBP < 140 mmHg
    if any([
            (x.target_variable != 'TAM') & (x.target_value == '140'),
            (x.target_variable != 'TAM') & ('-' in x.target_value) & (x.target_value.endswith('140')),
            ('140/' in x.target_value)
    ]):
        return 1

    # strategy 2: SBP < 150 mmHg
    if any([
            (x.target_variable != 'TAM') & (x.target_value == '150'),
            (x.target_variable != 'TAM') & ('-' in x.target_value) & (x.target_value.endswith('150')),
            ('150/' in x.target_value)
    ]):
        return 2

    # strategy 3: SBP < 160 mmHg
    if any([
            (x.target_variable != 'TAM') & (x.target_value == '160'),
            (x.target_variable != 'TAM') & ('-' in x.target_value) & (x.target_value.endswith('160')),
            ('160/' in x.target_value)
    ]):
        return 3

    # strategy 4: MAP < 130 mmHg
    if any([
            (x.target_variable == 'TAM') & (x.target_value == '130'),
            (x.target_variable == 'TAM') & ('-' in x.target_value) & (x.target_value.endswith('130'))
    ]):
        return 4

    # strategy 5: SBP < 180 mmHg
    if any([
            (x.target_value == '180'),
            (x.target_variable != 'TAM') & ('-' in x.target_value) & (x.target_value.endswith('180')),
            ('180/' in x.target_value)
    ]):
        return 5

    # strategy 6: SBP < 220 mmHg
    
    if any([
            (x.target_value == '220'),
            (x.target_variable != 'TAM') & ('-' in x.target_value) & (x.target_value.endswith('220')),
            ('220/' in x.target_value)
    ]):
        return 6
    
    return None
    

In [ ]:
antihypertensives_bp_targets_df['target_strategy'] = antihypertensives_bp_targets_df.apply(target_strategy_identification, axis=1)

In [ ]:
antihypertensives_bp_targets_df.target_strategy.value_counts()

In [ ]:
antihypertensives_bp_targets_df = antihypertensives_bp_targets_df[antihypertensives_bp_targets_df.target_strategy.notnull()]

In [ ]:
antihypertensives_bp_targets_df = antihypertensives_bp_targets_df.drop_duplicates()

#### Format time

Current format: entry per prescription with date from to 
Target format: entry per time resolution 


In [ ]:
antihypertensives_bp_targets_df

In [ ]:
# for every line in prescription, create a line per interval (day, hour, 15 min...)

interval = 60 # in minutes
datatime_format = '%d.%m.%Y %H:%M'

def create_intervals(start_date, end_date, interval):
    intervals = []
    start_date = pd.to_datetime(start_date, format=datatime_format)
    end_date = pd.to_datetime(end_date, format=datatime_format)
    current_date = start_date
    # create an interval for every timestep until end_date (included only if at least half of the interval is included)
    while current_date < end_date - pd.Timedelta(minutes=interval/2):
        intervals.append(current_date)
        current_date = current_date + pd.Timedelta(minutes=interval)
    return pd.Series(intervals)


def get_prescription_end_date(end_date, stop_date):
    end_date = pd.to_datetime(end_date, format=datatime_format)
    stop_date = pd.to_datetime(stop_date, format=datatime_format)
    # return whichever is earlier
    if end_date < stop_date:
        return end_date
    else:
        return stop_date
    
    
def get_prescription_intervals(x, interval):
    prescription_end_date = get_prescription_end_date(x.end_date, x.stop_date)
    return create_intervals(x.start_date, prescription_end_date, interval)
    


In [ ]:
df2 = antihypertensives_bp_targets_df.apply(lambda x: get_prescription_intervals(x, interval), axis=1)

In [ ]:
df3 = antihypertensives_bp_targets_df.join(df2)

In [ ]:
df3 = df3.melt(id_vars=antihypertensives_bp_targets_df.columns, value_name='sample_date')

In [ ]:
df3.drop(columns=['variable'], inplace=True)
# drop NaT in sample_date
df3.dropna(subset=['sample_date'], inplace=True)

In [ ]:
df3.head(50)

Display strategy as a function of date

In [ ]:
# plot strategy as a function of date for temp
import matplotlib.pyplot as plt
import seaborn as sns

temp = df3[df3.case_admission_id == '307385_6473']

fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data=temp, x='sample_date', y='target_strategy', hue='target_strategy', ax=ax)

# test full function

In [ ]:
from geneva_stroke_unit_preprocessing.prescription_preprocessing.anti_hypertensive_strategy_extraction import \
    extract_anti_hypertensive_strategy

test_df = extract_anti_hypertensive_strategy(prescription_df)

In [ ]:
test_df.head(50)

#### Verify that there is a single strategy at a time per case_admission_id
- strategy should not overlap in time (defined as start_date to end_date or stop_date, whichever is earlier)
- if strategies overlap, keep the most aggressive one (0 > 1 > 2 > 3 > 4 > 5 > 6 > 7)

-> this should be done after assigning timebines

## BP support strategies

In [ ]:
bp_support_targets_df = bp_targets_df[bp_targets_df.short_name.isin(pd.concat([drug_class_lists_df.vasopressor, drug_class_lists_df.fluid]).dropna().values)]
bp_support_targets_df

In [ ]:
bp_support_targets_df.target_value.value_counts()